In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import scipy 
import pandas as pd
import shutil

C:\Users\azada\Anaconda3\envs\py37\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\azada\Anaconda3\envs\py37\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


# CremaD 

In [2]:
def getEmotionFileNameCrema(x):
    part = x.split('_')
    if part[2] == 'SAD':
        return 'SAD'
    elif part[2] == 'ANG':
        return 'ANG'
    elif part[2] == 'DIS':
        return 'DIS'
    elif part[2] == 'FEA':
        return 'FEA'
    elif part[2] == 'HAP':
        return 'HAP'
    elif part[2] == 'NEU':
        return 'NEU'
    else:
        return 'Unknown'

In [3]:
root = r"../data/cremad/AudioWAV"

In [4]:
df=pd.read_csv(r"../data/cremad/VideoDemographics.csv")
df.head(3)

,ActorID,Age,Sex,Race,Ethnicity
0,1001,51,Male,Caucasian,Not Hispanic
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic


In [5]:
files = listdir(root)
len(files)

7442

In [6]:
female_ids=[str(df.loc[i].iat[0]) for i in df.index if df.loc[i].iat[2]=="Female"]

In [7]:
def split(load_path,path_to_save, split_time = 1.5):
    data, sr = librosa.load(load_path, sr = 22050)
    split_data_length = int(split_time * sr)
    split_count = int(len(data)/split_data_length)
    for i in range(split_count):
        start_index = int(i*split_data_length)
        data_split = data[start_index: start_index +split_data_length]
        file_ending = '_' +str(i+1) +'.wav'
        final_path = path_to_save.replace('.wav', file_ending)
        librosa.output.write_wav(final_path, data_split, sr)

In [9]:
folder_to_save = r"../data/cremad/Audio1_5WAV"
file_names = []

files = listdir(root)
for file in files:
    emot = getEmotionFileNameCrema(file)
    split_file_name = file.split("_")
    if split_file_name[0] in female_ids:
        gendered_file_name = "F_" + emot+ "_"+ file
    else:
        gendered_file_name = "M_" +emot+ "_"+  file
    split(join(root,file),join(folder_to_save,gendered_file_name))

## RAVDESS

In [10]:
# Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
# Vocal channel (01 = speech, 02 = song).
# Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
# Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
# Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
# Repetition (01 = 1st repetition, 02 = 2nd repetition).
# Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [11]:
def getEmotionFileNameRAV(x):
    part = x.split('-')
    if part[2] == '01':
        return 'NEU'
    elif part[2] == '02':
        return 'CAL'
    elif part[2] == '03':
        return 'HAP'
    elif part[2] == '04':
        return 'SAD'
    elif part[2] == '05':
        return 'ANG'
    elif part[2] == '06':
        return 'FEA'
    elif part[2] == '07':
        return 'DIS'
    elif part[2] == '08':
        return 'SUR'
    else:
        return 'Unknown'

In [12]:
root = r"../data/RAVEDESS/Audio_Speech_Actors_01-24"
folders = listdir(root)
len(folders)

24

In [13]:
folder_to_save = r"../data/RAVEDESS/Audio1_5WAV"
file_names = []
lenghts = []
for folder in folders:
    curr_dir = join(root,folder)
    files = listdir(curr_dir)
    for file in files:
        emot = getEmotionFileNameRAV(file)
        split_file_name = file.split("-")
        #Female are EVEN
        if(int(split_file_name[-1][:-4])%2 == 0):
            gendered_file_name = "F_" +emot+ "_"+  file
        else:
            gendered_file_name = "M_" +emot+ "_"+  file
        split(join(root, folder, file),join(folder_to_save,gendered_file_name))


# SAVEE

In [14]:
# 'DC', 'JE', 'JK' and 'KL' are four male speakers recorded for the SAVEE database


# --------------------------------------
# Audio data 
# --------------------------------------
# Audio files consist of audio WAV files sampled at 44.1 kHz

# There are 15 sentences for each of the 7 emotion categories.
# The initial letter(s) of the file name represents the emotion class, and the following digits represent the sentence number.
# The letters 'a', 'd', 'f', 'h', 'n', 'sa' and 'su' represent 'anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness' and 'surprise' emotion classes respectively. 
# E.g., 'd03.wav' is the 3rd disgust sentence. 


In [15]:
def getEmotionFileNameSAVEE(str):
    if str.startswith( 'a' ):
        return 'ANG'
    elif str.startswith( 'd' ):
        return 'DIS'
    elif str.startswith( 'f' ):
        return 'FEA'
    elif str.startswith( 'h' ):
        return 'HAP'
    elif str.startswith( 'n' ):
        return 'NEU'
    elif str.startswith( 'sa' ):
        return 'SAD'
    elif str.startswith( 'su' ):
        return 'SUR'
    else:
        return 'Unknown'


In [22]:
root = r"../data/SAVEE/AudioData"
folders = listdir(root)
len(folders)

4

In [23]:
folder_to_save = r"../data/SAVEE/Audio1_5WAV"
for folder in folders:
    curr_dir = join(root,folder)
    files = listdir(curr_dir)
    print(folder,len(files))
    for file in files:
        emot = getEmotionFileNameSAVEE(file)
        gendered_file_name = "M_" +emot+ "_"+  file
        split(join(root, folder, file),join(folder_to_save,gendered_file_name))


DC 120
JE 120
JK 120
KL 120


# Tess

In [24]:
# There are a set of 200 target words were spoken in the carrier phrase "Say the word _' by two actresses (aged 26 and 64 years) and recordings were made of the set portraying each of seven emotions (anger, disgust, fear, happiness, pleasant surprise, sadness, and neutral). There are 2800 data points (audio files) in total.

# The dataset is organised such that each of the two female actor and their emotions are contain within its own folder. And within that, all 200 target words audio file can be found. The format of the audio file is a WAV format

In [25]:
def getEmotionFileNameTESS(x):
    part = x.split('_')
    part[2] = part[2][:-4]
    if part[2] == 'angry':
        return 'ANG'
    elif part[2] == 'disgust':
        return 'DIS'
    elif part[2] == 'fear':
        return 'FEA'
    elif part[2] == 'happy':
        return 'HAP'
    elif part[2] == 'neutral':
        return 'NEU'
    elif part[2] == 'ps':
        return 'SUR'
    elif part[2] == 'sad':
        return 'SAD'
    else:
        return 'Unknown'

In [26]:
root = r"../data/TESS/speech_data"
folders = listdir(root)
len(folders)

14

In [27]:
folder_to_save = r"../data/TESS/Audio1_5WAV"
for folder in folders:
    curr_dir = join(root,folder)
    files = listdir(curr_dir)
    for file in files:
        emot = getEmotionFileNameTESS(file)
        gendered_file_name = "F_" +emot+ "_"+  file
        split(join(root, folder, file),join(folder_to_save,gendered_file_name))